In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score



In [9]:
import pandas as pd
df=pd.read_csv("C:/Users/User/Desktop/students.csv")
print(df)


    Study_Hours  Attendance  Previous_Score  Final_Percentage
0             7          94              58         79.565250
1             4          96              59         72.371796
2             8          73              71         76.520927
3             5          62              46         54.153296
4             7          60              80         68.818388
..          ...         ...             ...               ...
95            3          79              44         54.695747
96            8          95              87         87.264887
97            6          78              58         64.797853
98            3          85              43         60.430899
99            1          62              74         47.509924

[100 rows x 4 columns]


In [10]:
df.isnull().sum()

Study_Hours         0
Attendance          0
Previous_Score      0
Final_Percentage    0
dtype: int64

In [11]:
df[df.duplicated()]

,Study_Hours,Attendance,Previous_Score,Final_Percentage


In [21]:
df['Study_Hours'].fillna(df['Study_Hours'].mean(),inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_13480\715074859.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Study_Hours'].fillna(df['Study_Hours'].mean(),inplace=True)


In [12]:
df.drop_duplicates(inplace=True)

In [19]:
import pandas as pd
#To find in which columns having outliers  in the dataframe
# Function to detect outlier columns using IQR
def find_outlier_columns_iqr(df):
    outlier_columns = []
    for col in df.select_dtypes(include='number').columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        outliers = df[(df[col] < lower) | (df[col] > upper)]
        if not outliers.empty:
            outlier_columns.append(col)
    return outlier_columns

outlier_cols = find_outlier_columns_iqr(df)
print("Columns with outliers:", outlier_cols)


Columns with outliers: []


In [20]:
#if the outliers are present<5 we have to drop those otherwise we have clip the outliers
df['Study_Hours']=df['Study_Hours'].clip(1,15)

In [13]:
df['Study_Attendance']=df['Study_Hours']*df['Attendance']

In [14]:
df

,Study_Hours,Attendance,Previous_Score,Final_Percentage,Study_Attendance
0,7,94,58,79.565250,658
1,4,96,59,72.371796,384
2,8,73,71,76.520927,584
3,5,62,46,54.153296,310
4,7,60,80,68.818388,420
...,...,...,...,...,...
95,3,79,44,54.695747,237
96,8,95,87,87.264887,760
97,6,78,58,64.797853,468
98,3,85,43,60.430899,255


In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
Features=['Study_Hours','Attendance','Previous_Score','Study_Attendance']
x=df[Features]
y=df['Final_Percentage'] 

In [16]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [17]:
scaler=StandardScaler()
x_train_scaled=scaler.fit_transform(x_train)
x_test_scaled=scaler.transform(x_test)

In [19]:
from sklearn.neighbors import KNeighborsRegressor

# Create the model
knn_model = KNeighborsRegressor(n_neighbors=5,weights='distance',
                              metric='euclidean')

# Fit the model to the training data
knn_model.fit(x_train_scaled, y_train)

                             

KNeighborsRegressor(metric='euclidean', weights='distance')

In [21]:
param_grid={'n_neighbors':[3,5,7,10],
            'weights':['uniform','distance'],
            'metric':['euclidean','manhattan']}
grid_search=GridSearchCV(knn_model,param_grid,cv=5,scoring='neg_mean_squared_error')
grid_search.fit(x_train_scaled,y_train)
            

GridSearchCV(cv=5,
             estimator=KNeighborsRegressor(metric='euclidean',
                                           weights='distance'),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': [3, 5, 7, 10],
                         'weights': ['uniform', 'distance']},
             scoring='neg_mean_squared_error')

In [24]:
best_knn = grid_search.best_estimator_
print("Best parameters:", grid_search.best_params_)


Best parameters: {'metric': 'euclidean', 'n_neighbors': 7, 'weights': 'distance'}


In [26]:
y_pred=best_knn.predict(x_test_scaled)

In [33]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R² Score: {r2:.2f}")


Mean Absolute Error (MAE): 2.91
Root Mean Squared Error (RMSE): 3.63
R² Score: 0.90


In [34]:
new_student=[[8,90,85,8*90]]
new_student_scaled=scaler.transform(new_student)
prediction=best_knn.predict(new_student_scaled)[0]
print(f"predicted percentage:{prediction:.2f}")

predicted percentage:86.27


C:\ProgramData\anaconda\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
